# SQL 的五十道練習

> 垂直與水平合併查詢結果

[數據交點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
%LOAD ../databases/imdb.db

In [2]:
ATTACH "../databases/nba.db" AS nba;

In [3]:
ATTACH "../databases/twElection2020.db" AS twElection2020;

In [4]:
ATTACH "../databases/covid19.db" AS covid19;

## 垂直合併查詢結果

## 在「包含子查詢的結構 」章節中，我們開始看過一段 SQL 中包含著另一段 SQL 的使用方法

- 將多個 `SELECT ... FROM ...` 結合至一段 SQL 中。
- 能夠結合不同資料表中的資料到一個查詢結果中。

## 從學會子查詢開始，我們開始能將查詢的範圍由「單個」資料來源延展至「多個」資料來源。

## 以 `UNION` 敘述垂直合併多個 `SELECT ... FROM ...`

```sql
-- UNION 敘述
SELECT column_name
  FROM table_name
 UNION
SELECT column_name
  FROM table_name;
```

In [5]:
SELECT firstName,
       lastName,
       weightKilograms AS weight,
       'kilograms' AS unit
  FROM players
 WHERE firstName = 'LeBron' AND
       lastName = 'James'
 UNION
SELECT firstName,
       lastName,
       weightPounds AS weight,
       'pounds' AS unit
  FROM players
 WHERE firstName = 'LeBron' AND
       lastName = 'James';

firstName,lastName,weight,unit
LeBron,James,113.4,kilograms
LeBron,James,250.0,pounds


## 使用 `UNION` 敘述時的注意事項

- 垂直合併的查詢結果欄位個數要相同。
- 垂直合併的查詢中若有 `ORDER BY` 只能放在 `UNION` 之後。
- 垂直合併的查詢結果若有重複的觀測值會被刪除。

In [6]:
-- 垂直合併的查詢結果欄位個數不相同。
SELECT firstName,
       lastName,
       weightKilograms AS weight,
       'kilograms' AS unit
  FROM players
 WHERE firstName = 'LeBron' AND
       lastName = 'James'
 UNION
SELECT firstName,
       lastName,
       weightPounds AS weight
  FROM players
 WHERE firstName = 'LeBron' AND
       lastName = 'James';

Error: SELECTs to the left and right of UNION do not have the same number of result columns

In [7]:
-- 垂直合併的查詢中若有 ORDER BY 只能放在 UNION 之後。
SELECT firstName,
       lastName,
       weightKilograms AS weight,
       'kilograms' AS unit
  FROM players
 WHERE lastName = 'James'
 ORDER BY weightKilograms
 UNION
SELECT firstName,
       lastName,
       weightPounds AS weight,
       'pounds' AS unit
  FROM players
 WHERE lastName = 'James';

Error: ORDER BY clause should come after UNION not before

In [8]:
-- 垂直合併的查詢結果若有重複的觀測值會被刪除。
SELECT firstName,
       lastName,
       weightKilograms AS weight,
       'kilograms' AS unit
  FROM players
 WHERE firstName = 'LeBron' AND
       lastName = 'James'
 UNION
SELECT firstName,
       lastName,
       weightKilograms AS weight,
       'kilograms' AS unit
  FROM players
 WHERE firstName = 'LeBron' AND
       lastName = 'James';

firstName,lastName,weight,unit
LeBron,James,113.4,kilograms


## 若是希望保留重複觀測值，改使用 `UNION ALL` 敘述

```sql
-- UNION ALL 敘述
SELECT column_name
  FROM table_name
 UNION ALL
SELECT column_name
  FROM table_name;
```

In [9]:
-- UNION ALL 敘述
SELECT firstName,
       lastName,
       weightKilograms AS weight,
       'kilograms' AS unit
  FROM players
 WHERE firstName = 'LeBron' AND
       lastName = 'James'
 UNION ALL
SELECT firstName,
       lastName,
       weightKilograms AS weight,
       'kilograms' AS unit
  FROM players
 WHERE firstName = 'LeBron' AND
       lastName = 'James';

firstName,lastName,weight,unit
LeBron,James,113.4,kilograms
LeBron,James,113.4,kilograms


## 水平合併查詢結果

## 在「垂直合併查詢結果」單元中，我們使用 `UNION` 或 `UNION ALL` 的敘述沿著觀測值方向（垂直方向）增加資料。

## 水平合併查詢結果則是能夠讓我們使用 `JOIN` 敘述沿著欄位方向（水平方向）增加資料

因為是「水平」合併，在 `FROM` 敘述後的資料表被稱為「左表格」、`JOIN` 敘述後的資料表被稱為「右表格」。

```sql
-- JOIN 敘述
SELECT left_table.column_name,
       right_table.column_name
  FROM table_name AS left_table
  JOIN table_name AS right_table
    ON left_table.join_key = right_table.join_key;
```

In [10]:
-- 使用 JOIN 敘述連結 players 與 teams 資料表的欄位
SELECT teams.fullName,
       players.firstName,
       players.lastName
  FROM teams
  JOIN players
    ON teams.teamId = players.teamId
 WHERE teams.nickname = 'Lakers';

fullName,firstName,lastName
Los Angeles Lakers,LeBron,James
Los Angeles Lakers,Jared,Dudley
Los Angeles Lakers,Marc,Gasol
Los Angeles Lakers,Wesley,Matthews
Los Angeles Lakers,Markieff,Morris
Los Angeles Lakers,Anthony,Davis
Los Angeles Lakers,Dennis,Schroder
Los Angeles Lakers,Kentavious,Caldwell-Pope
Los Angeles Lakers,Montrezl,Harrell
Los Angeles Lakers,Quinn,Cook


In [11]:
-- 使用 JOIN 敘述連結 top_rated_movies, casting 與 actors 資料表的欄位
SELECT movies.title,
       casting.ord,
       actors.name
  FROM top_rated_movies AS movies
  JOIN casting
    ON movies.id = casting.movie_id
  JOIN actors
    ON casting.actor_id = actors.id
 WHERE movies.title = 'The Shawshank Redemption';

title,ord,name
The Shawshank Redemption,1,Tim Robbins
The Shawshank Redemption,2,Morgan Freeman
The Shawshank Redemption,3,Bob Gunton
The Shawshank Redemption,4,William Sadler
The Shawshank Redemption,5,Clancy Brown
The Shawshank Redemption,6,Gil Bellows
The Shawshank Redemption,7,Mark Rolston
The Shawshank Redemption,8,James Whitmore
The Shawshank Redemption,9,Jeffrey DeMunn
The Shawshank Redemption,10,Larry Brandenburg


## 使用 `JOIN` 敘述時的注意事項

- 養成為水平合併的資料表取別名並在欄位名稱前註明的好習慣。
- 除了實體資料表亦可使用子查詢作為合併來源。
- 預設保留合併資料所交集的觀測值。

## 養成為水平合併的資料表取別名並在欄位名稱前註明的好習慣，舉例來說，在 `top_rated_movies` 與 `actors` 兩個資料表中都有 `id` 欄位

- `top_rated_movies.id` 指的是電影編號。
- `actors.id` 指的是演員編號。

In [12]:
-- 沒有指定 id 來自哪一個資料表
SELECT title,
       ord,
       name
  FROM top_rated_movies AS movies
  JOIN casting
    ON id = movie_id
  JOIN actors
    ON actor_id = id
 WHERE title = 'The Shawshank Redemption';

Error: ambiguous column name: id

## 除了實體資料表亦可使用子查詢作為合併來源，舉例來說，讓左表格與右表格都是子查詢。

In [13]:
-- 除了實體資料表亦可使用子查詢作為合併來源。
SELECT team_lakers.fullName,
       players.firstName,
       players.lastName
  FROM (SELECT teamId,
               fullName
          FROM teams
         WHERE nickname = 'Lakers') AS team_lakers
  JOIN (SELECT firstName,
               lastName,
               teamId
          FROM players) AS players
    ON team_lakers.teamId = players.teamId;

fullName,firstName,lastName
Los Angeles Lakers,LeBron,James
Los Angeles Lakers,Jared,Dudley
Los Angeles Lakers,Marc,Gasol
Los Angeles Lakers,Wesley,Matthews
Los Angeles Lakers,Markieff,Morris
Los Angeles Lakers,Anthony,Davis
Los Angeles Lakers,Dennis,Schroder
Los Angeles Lakers,Kentavious,Caldwell-Pope
Los Angeles Lakers,Montrezl,Harrell
Los Angeles Lakers,Quinn,Cook


## 預設保留合併資料所交集的觀測值，舉例來說，左表格是洛杉磯湖人隊與波士頓賽爾提克隊的隊伍資料，右表格是洛杉磯湖人隊與費城七六人隊的球員資料。

In [14]:
-- 預設保留合併資料所交集的觀測值：洛杉磯湖人隊的隊伍與球員資料。
SELECT teams.fullName,
       players.firstName,
       players.lastName
  FROM (SELECT teamId,
               fullName
          FROM teams
         WHERE nickname IN ('Lakers', 'Celtics')) AS teams -- 左表格是洛杉磯湖人隊與波士頓賽爾提克隊的隊伍資料
  JOIN (SELECT firstName,
               lastName,
               teamId
          FROM players
         WHERE teamId IN (1610612747, 1610612755)) AS players -- 右表格是洛杉磯湖人隊與費城七六人隊的球員資料
    ON teams.teamId = players.teamId;

fullName,firstName,lastName
Los Angeles Lakers,LeBron,James
Los Angeles Lakers,Jared,Dudley
Los Angeles Lakers,Marc,Gasol
Los Angeles Lakers,Wesley,Matthews
Los Angeles Lakers,Markieff,Morris
Los Angeles Lakers,Anthony,Davis
Los Angeles Lakers,Dennis,Schroder
Los Angeles Lakers,Kentavious,Caldwell-Pope
Los Angeles Lakers,Montrezl,Harrell
Los Angeles Lakers,Quinn,Cook


## 若是希望保留左表格為主的觀測值，改使用 `LEFT JOIN` 敘述

```sql
-- LEFT JOIN 敘述
SELECT left_table.column_name,
       right_table.column_name
  FROM table_name AS left_table
  LEFT JOIN table_name AS right_table
    ON left_table.join_key = right_table.join_key;
```

In [15]:
-- 預設保留左表格為主的觀測值：洛杉磯湖人隊與波士頓賽爾提克隊的隊伍資料、洛杉磯湖人隊的球員資料。
SELECT teams.fullName,
       players.firstName,
       players.lastName
  FROM (SELECT teamId,
               fullName
          FROM teams
         WHERE nickname IN ('Lakers', 'Celtics')) AS teams -- 左表格是洛杉磯湖人隊與波士頓賽爾提克隊的隊伍資料
  LEFT JOIN (SELECT firstName,
                    lastName,
                    teamId
               FROM players
              WHERE teamId IN (1610612747, 1610612755)) AS players -- 右表格是洛杉磯湖人隊與費城七六人隊的球員資料
    ON teams.teamId = players.teamId;

fullName,firstName,lastName
Boston Celtics,,
Los Angeles Lakers,Alex,Caruso
Los Angeles Lakers,Alfonzo,McKinnie
Los Angeles Lakers,Anthony,Davis
Los Angeles Lakers,Dennis,Schroder
Los Angeles Lakers,Devontae,Cacok
Los Angeles Lakers,Jared,Dudley
Los Angeles Lakers,Kentavious,Caldwell-Pope
Los Angeles Lakers,Kostas,Antetokounmpo
Los Angeles Lakers,Kyle,Kuzma


## 重點統整

- 將查詢的範圍由「單個」資料來源延展至「多個」資料來源。
    - 子查詢。
    - 垂直合併：`UNION` 敘述與 `UNION ALL` 敘述。
    - 水平合併：`JOIN` 敘述與 `LEFT JOIN` 敘述。

## 目前我們會的 SQL

```sql
-- 可以在多個敘述後結合子查詢
SELECT column_name
  FROM table_name AS left_table
  JOIN table_name AS right_table
    ON left_table.join_key = right_table.join_key
 WHERE condition
 GROUP BY column_name
HAVING condition
 UNION 
SELECT ... 
  FROM ...
 ORDER BY column_name
 LIMIT m;
```